In [3]:
import numpy as np
import pandas as pd
import json
import pprint
import json
pp = pprint.PrettyPrinter(indent=4)
pd.options.display.max_colwidth = 100

In [4]:
def df_mean(path, apply_to_df):
    df_mean = pd.read_csv(path)
    df_mean = apply_to_df(df_mean)
        
    if 'nb_movies_not_in_train' in df_mean:
        df_mean = df_mean.drop(['nb_movies_not_in_train'],axis=1)
    if 'nb_test_ratings' in df_mean:
        df_mean = df_mean.drop(['nb_test_ratings'],axis=1)
    if 'nb_train_ratings' in df_mean:
        df_mean = df_mean.drop(['nb_train_ratings'],axis=1)
    if 'user_id' in df_mean:
        df_mean = df_mean.drop(['user_id'],axis=1)
    
    df_mean = df_mean.mean(0).to_frame().T
    df_mean['path'] = path
    
    return df_mean
    

def get_df_results(results, max_train_ratings=None):
    if not max_train_ratings:
        apply_to_df = lambda df: df[df['nb_test_ratings'] > 0]
    else:
        apply_to_df = lambda df: df[(df['nb_test_ratings'] > 0) & (df['nb_train_ratings'] <= max_train_ratings)]
        
    result_map = map(lambda path: df_mean(path, apply_to_df), results)
    return reduce(lambda x, y: x.append(y), result_map)

## BEST ML-100k - 0.7 train / 0.3 test - sparse item

In [ ]:
user_metrics = ['../metrics/mpcf/2016-05-27_20.57.43_no-si_ml-1m_e20_tt-0.7_task-22_user-metrics.csv']
df_results = get_df_results(user_metrics)
df_results.sort_values('auc', ascending=False)

In [ ]:
# best config
with open('../models/mpcf/2016-06-20_18.30.50_no-si_ml-100k_e20_tt-0.7_base_config.json') as f:
    pp.pprint(json.loads(f.read()))

In [ ]:
movie_metrics = ['../metrics/mpcf/2016-06-20_18.30.50_no-si_ml-100k_e20_tt-0.7_base_movie-metrics.csv']
df_results = get_df_results(movie_metrics)
df_results.sort_values('auc', ascending=False)

# Gridsearch

In [12]:
user_metrics = [
'../metrics/mfnn/2016-06-20_14.41.10_mfnn_ml-100k_e20_tt-0.7_task-0_user-metrics.csv',
'../metrics/mfnn/2016-06-20_15.23.33_mfnn_ml-100k_e20_tt-0.7_task-0_user-metrics.csv',
'../metrics/mfnn/2016-06-20_15.27.24_mfnn_ml-100k_e20_tt-0.7_task-0_user-metrics.csv',
'../metrics/mfnn/2016-06-20_15.35.53_mfnn_ml-100k_e20_tt-0.7_task-0_user-metrics.csv',
'../metrics/mfnn/2016-06-20_15.57.48_mfnn_ml-100k_e20_tt-0.7_task-0_user-metrics.csv',
'../metrics/mfnn/2016-06-20_16.59.18_mfnn_ml-100k_e20_tt-0.7_task-1_user-metrics.csv',
'../metrics/mfnn/2016-06-20_17.56.21_mfnn_ml-100k_e20_tt-0.7_task-1_user-metrics.csv',
'../metrics/mfnn/2016-06-20_18.03.09_mfnn_ml-100k_e20_tt-0.7_task-1_user-metrics.csv',
'../metrics/mfnn/2016-06-20_19.02.32_mfnn_ml-100k_e20_tt-0.7_task-2_user-metrics.csv',
'../metrics/mfnn/2016-06-20_20.07.00_mfnn_ml-100k_e20_tt-0.7_task-2_user-metrics.csv',
'../metrics/mfnn/2016-06-20_20.14.07_mfnn_ml-100k_e20_tt-0.7_task-2_user-metrics.csv',
'../metrics/mfnn/2016-06-20_22.08.26_mfnn_ml-100k_e20_tt-0.7_task-3_user-metrics.csv',
'../metrics/mfnn/2016-06-20_22.35.49_mfnn_ml-100k_e20_tt-0.7_task-3_user-metrics.csv',
'../metrics/mfnn/2016-06-21_03.33.28_mfnn_ml-100k_e20_tt-0.7_task-1_user-metrics.csv',
'../metrics/mfnn/2016-06-21_04.38.48_mfnn_ml-100k_e20_tt-0.7_task-0_user-metrics.csv',
'../metrics/mfnn/2016-06-21_06.21.30_mfnn_ml-100k_e20_tt-0.7_task-1_user-metrics.csv',
'../metrics/mfnn/2016-06-21_08.17.14_mfnn_ml-100k_e20_tt-0.7_task-2_user-metrics.csv',
'../metrics/mfnn/2016-06-21_09.15.54_mfnn_ml-100k_e20_tt-0.7_task-3_user-metrics.csv',
'../metrics/mfnn/2016-06-21_10.20.59_mfnn_ml-100k_e20_tt-0.7_task-3_user-metrics.csv',
'../metrics/mfnn/2016-06-21_13.27.13_mfnn_ml-100k_e20_tt-0.7_task-2_user-metrics.csv',
]

In [13]:
df_results = get_df_results(user_metrics)
df_results.sort_values('auc', ascending=False)

,auc,avg_precision,f1,fcp,precision_at_20,recall_at_20,reciprocal_rank,spearman_rank_corr,path
0,0.933496,0.306996,0.269600,0.715244,0.266702,0.415502,0.637832,0.190412,../metrics/mfnn/2016-06-20_22.08.26_mfnn_ml-100k_e20_tt-0.7_task-3_user-metrics.csv
0,0.929596,0.282412,0.250763,0.714033,0.249099,0.385689,0.612299,0.186699,../metrics/mfnn/2016-06-20_16.59.18_mfnn_ml-100k_e20_tt-0.7_task-1_user-metrics.csv
0,0.929300,0.351118,0.294186,0.745600,0.292100,0.450057,0.702588,0.280343,../metrics/mfnn/2016-06-21_03.33.28_mfnn_ml-100k_e20_tt-0.7_task-1_user-metrics.csv
0,0.928658,0.278104,0.249006,0.713842,0.247720,0.382489,0.605962,0.187090,../metrics/mfnn/2016-06-20_15.23.33_mfnn_ml-100k_e20_tt-0.7_task-0_user-metrics.csv
0,0.928464,0.353380,0.292917,0.746260,0.291039,0.446157,0.707856,0.282339,../metrics/mfnn/2016-06-21_08.17.14_mfnn_ml-100k_e20_tt-0.7_task-2_user-metrics.csv
0,0.928007,0.320823,0.273305,0.744593,0.272747,0.414704,0.675805,0.277141,../metrics/mfnn/2016-06-20_15.27.24_mfnn_ml-100k_e20_tt-0.7_task-0_user-metrics.csv
0,0.926705,0.314472,0.268958,0.744357,0.268346,0.408378,0.666743,0.276841,../metrics/mfnn/2016-06-21_06.21.30_mfnn_ml-100k_e20_tt-0.7_task-1_user-metrics.csv
0,0.925691,0.303821,0.260660,0.743365,0.259650,0.398210,0.654882,0.273701,../metrics/mfnn/2016-06-20_22.35.49_mfnn_ml-100k_e20_tt-0.7_task-3_user-metrics.csv
0,0.924101,0.295554,0.256256,0.741017,0.255037,0.392301,0.638360,0.266473,../metrics/mfnn/2016-06-21_13.27.13_mfnn_ml-100k_e20_tt-0.7_task-2_user-metrics.csv
0,0.920026,0.254347,0.233028,0.712977,0.235207,0.349612,0.581458,0.183622,../metrics/mfnn/2016-06-21_10.20.59_mfnn_ml-100k_e20_tt-0.7_task-3_user-metrics.csv


In [14]:
# best config
with open('../models/mfnn/2016-06-20_22.08.26_mfnn_ml-100k_e20_tt-0.7_task-3_config.json') as f:
    pp.pprint(json.loads(f.read()))

{   u'ada_eps': 1e-06,
    u'adagrad': True,
    u'binarize': True,
    u'binarize_neg': 0,
    u'binarize_pos': 1,
    u'binarize_threshold': 1,
    u'd2v_model': u'doc2vec-models/2016-04-14_17.36.08_20e_pv-dbow_size50_lr0.025_window8_neg5',
    u'eval_in_parallel': True,
    u'experiment_name': u'mfnn_ml-100k_e20_tt-0.7_task-3',
    u'hit_threshold': 4,
    u'lr': 0.015,
    u'lr_decay': 0.02,
    u'metrics_save_dir': u'metrics/mfnn',
    u'model_save_dir': u'models/mfnn',
    u'nb_d2v_features': 50,
    u'nb_epochs': 20,
    u'nb_latent_f': 128,
    u'nb_movies': 1261,
    u'nb_users': 943,
    u'pool_size': 24,
    u'precision_recall_at_n': 20,
    u'ratings_path': u'data/splits/ml-100k/ratings.csv',
    u'reg_lambda': 0.01,
    u'run_eval': True,
    u'run_movie_metrics': True,
    u'sparse_item': True,
    u'test': True,
    u'test_path': u'data/splits/ml-100k/sparse-item/0.7-test.csv',
    u'top_n_predictions': 100,
    u'train_path': u'data/splits/ml-100k/sparse-item/0.7-train.

## ML-100k - 0.7 train / 0.3 test - sparse item

#### User Metrics

In [7]:
user_metrics = [
'../metrics/mfnn/2016-05-27_20.32.35_mfnn_ml-100k_e20_tt-0.7_task-19_user-metrics.csv',
'../metrics/mfnn/2016-05-27_20.33.16_mfnn_ml-100k_e20_tt-0.7_task-6_user-metrics.csv',
'../metrics/mfnn/2016-05-27_20.40.31_mfnn_ml-100k_e20_tt-0.7_task-8_user-metrics.csv',
'../metrics/mfnn/2016-05-27_21.23.18_mfnn_ml-100k_e20_tt-0.7_task-3_user-metrics.csv',
'../metrics/mfnn/2016-05-27_21.36.57_mfnn_ml-100k_e20_tt-0.7_task-0_user-metrics.csv',
'../metrics/mfnn/2016-05-27_22.01.54_mfnn_ml-100k_e20_tt-0.7_task-12_user-metrics.csv',
'../metrics/mfnn/2016-05-27_22.07.28_mfnn_ml-100k_e20_tt-0.7_task-5_user-metrics.csv',
'../metrics/mfnn/2016-05-27_22.14.10_mfnn_ml-100k_e20_tt-0.7_task-9_user-metrics.csv',
'../metrics/mfnn/2016-05-27_23.32.04_mfnn_ml-100k_e20_tt-0.7_task-1_user-metrics.csv',
'../metrics/mfnn/2016-05-28_00.07.22_mfnn_ml-100k_e20_tt-0.7_task-7_user-metrics.csv',
'../metrics/mfnn/2016-05-28_00.26.03_mfnn_ml-100k_e20_tt-0.7_task-10_user-metrics.csv',
'../metrics/mfnn/2016-05-28_01.26.12_mfnn_ml-100k_e20_tt-0.7_task-11_user-metrics.csv',
'../metrics/mfnn/2016-05-28_02.29.58_mfnn_ml-100k_e20_tt-0.7_task-23_user-metrics.csv',
'../metrics/mfnn/2016-05-28_02.32.17_mfnn_ml-100k_e20_tt-0.7_task-18_user-metrics.csv',
'../metrics/mfnn/2016-05-28_03.04.40_mfnn_ml-100k_e20_tt-0.7_task-17_user-metrics.csv',
'../metrics/mfnn/2016-05-28_04.07.27_mfnn_ml-100k_e20_tt-0.7_task-2_user-metrics.csv',
'../metrics/mfnn/2016-05-30_17.43.10_mfnn_ml-100k_e20_tt-0.7_task-23_user-metrics.csv',
'../metrics/mfnn/2016-05-30_17.44.44_mfnn_ml-100k_e20_tt-0.7_task-7_user-metrics.csv',
'../metrics/mfnn/2016-05-30_17.45.10_mfnn_ml-100k_e20_tt-0.7_task-15_user-metrics.csv',
'../metrics/mfnn/2016-05-30_17.49.49_mfnn_ml-100k_e20_tt-0.7_task-3_user-metrics.csv',
'../metrics/mfnn/2016-05-30_18.03.38_mfnn_ml-100k_e20_tt-0.7_task-12_user-metrics.csv',
'../metrics/mfnn/2016-05-30_18.09.38_mfnn_ml-100k_e20_tt-0.7_task-19_user-metrics.csv',
'../metrics/mfnn/2016-05-30_18.23.41_mfnn_ml-100k_e20_tt-0.7_task-4_user-metrics.csv',
'../metrics/mfnn/2016-05-30_18.27.02_mfnn_ml-100k_e20_tt-0.7_task-2_user-metrics.csv',
'../metrics/mfnn/2016-05-30_18.34.38_mfnn_ml-100k_e20_tt-0.7_task-0_user-metrics.csv',
'../metrics/mfnn/2016-05-30_18.35.21_mfnn_ml-100k_e20_tt-0.7_task-16_user-metrics.csv',
'../metrics/mfnn/2016-05-30_18.35.26_mfnn_ml-100k_e20_tt-0.7_task-11_user-metrics.csv',
'../metrics/mfnn/2016-05-30_18.55.22_mfnn_ml-100k_e20_tt-0.7_task-8_user-metrics.csv',
'../metrics/mfnn/2016-05-30_19.03.48_mfnn_ml-100k_e20_tt-0.7_task-1_user-metrics.csv',
'../metrics/mfnn/2016-05-30_19.15.45_mfnn_ml-100k_e20_tt-0.7_task-6_user-metrics.csv',
'../metrics/mfnn/2016-05-30_19.20.26_mfnn_ml-100k_e20_tt-0.7_task-5_user-metrics.csv',
'../metrics/mfnn/2016-05-30_20.39.37_mfnn_ml-100k_e20_tt-0.7_task-17_user-metrics.csv',
'../metrics/mfnn/2016-05-31_02.58.20_mfnn_ml-100k_e20_tt-0.7_task-9_user-metrics.csv',
'../metrics/mfnn/2016-05-31_05.02.41_mfnn_ml-100k_e20_tt-0.7_task-22_user-metrics.csv',
'../metrics/mfnn/2016-05-31_07.04.38_mfnn_ml-100k_e20_tt-0.7_task-18_user-metrics.csv',
'../metrics/mfnn/2016-05-31_10.53.16_mfnn_ml-100k_e20_tt-0.7_task-10_user-metrics.csv',
'../metrics/mfnn/2016-05-31_11.40.24_mfnn_ml-100k_e20_tt-0.7_task-14_user-metrics.csv',
]

In [8]:
df_results = get_df_results(user_metrics)
df_results.sort_values('auc', ascending=False)

,auc,avg_precision,f1,fcp,precision_at_20,recall_at_20,reciprocal_rank,spearman_rank_corr,path
0,0.934452,0.318685,0.278498,0.715248,0.275928,0.424804,0.644229,0.191657,../metrics/mfnn/2016-05-31_10.53.16_mfnn_ml-100k_e20_tt-0.7_task-10_user-metrics.csv
0,0.932942,0.341954,0.289460,0.716163,0.286214,0.441671,0.681852,0.193801,../metrics/mfnn/2016-05-30_19.15.45_mfnn_ml-100k_e20_tt-0.7_task-6_user-metrics.csv
0,0.932724,0.311204,0.271850,0.715243,0.268664,0.416425,0.637425,0.191073,../metrics/mfnn/2016-05-31_07.04.38_mfnn_ml-100k_e20_tt-0.7_task-18_user-metrics.csv
0,0.932351,0.296824,0.261461,0.712774,0.257529,0.403653,0.625815,0.186020,../metrics/mfnn/2016-05-27_22.14.10_mfnn_ml-100k_e20_tt-0.7_task-9_user-metrics.csv
0,0.931380,0.300129,0.260716,0.715281,0.256204,0.405328,0.635825,0.190341,../metrics/mfnn/2016-05-30_18.34.38_mfnn_ml-100k_e20_tt-0.7_task-0_user-metrics.csv
0,0.928385,0.284894,0.251215,0.711874,0.248356,0.391072,0.613613,0.179900,../metrics/mfnn/2016-05-30_19.03.48_mfnn_ml-100k_e20_tt-0.7_task-1_user-metrics.csv
0,0.924877,0.322756,0.273609,0.747049,0.271368,0.420512,0.668998,0.285874,../metrics/mfnn/2016-05-27_23.32.04_mfnn_ml-100k_e20_tt-0.7_task-1_user-metrics.csv
0,0.924349,0.324539,0.275348,0.747511,0.272428,0.421840,0.669442,0.287308,../metrics/mfnn/2016-05-27_20.33.16_mfnn_ml-100k_e20_tt-0.7_task-6_user-metrics.csv
0,0.923276,0.323844,0.273868,0.746172,0.271580,0.420249,0.677415,0.281257,../metrics/mfnn/2016-05-28_02.32.17_mfnn_ml-100k_e20_tt-0.7_task-18_user-metrics.csv
0,0.923042,0.322102,0.272084,0.748716,0.270520,0.417302,0.674369,0.288675,../metrics/mfnn/2016-05-28_03.04.40_mfnn_ml-100k_e20_tt-0.7_task-17_user-metrics.csv


In [6]:
# best config
with open('../models/mfnn/2016-05-31_10.53.16_mfnn_ml-100k_e20_tt-0.7_task-10_config.json') as f:
    pp.pprint(json.loads(f.read()))

{   u'ada_eps': 1e-06,
    u'adagrad': True,
    u'binarize': True,
    u'binarize_neg': 0,
    u'binarize_pos': 1,
    u'binarize_threshold': 1,
    u'd2v_model': u'doc2vec-models/2016-04-14_17.36.08_20e_pv-dbow_size50_lr0.025_window8_neg5',
    u'eval_in_parallel': False,
    u'experiment_name': u'mfnn_ml-100k_e20_tt-0.7_task-10',
    u'hit_threshold': 4,
    u'lr': 0.03,
    u'lr_decay': 0.02,
    u'metrics_save_dir': u'metrics/mfnn',
    u'model_save_dir': u'models/mfnn',
    u'nb_d2v_features': 50,
    u'nb_epochs': 20,
    u'nb_latent_f': 96,
    u'nb_movies': 1261,
    u'nb_users': 943,
    u'precision_recall_at_n': 20,
    u'ratings_path': u'data/splits/ml-100k/ratings.csv',
    u'reg_lambda': 0.01,
    u'run_eval': True,
    u'run_movie_metrics': False,
    u'sparse_item': True,
    u'test': True,
    u'test_path': u'data/splits/ml-100k/sparse-item/0.7-test.csv',
    u'top_n_predictions': 100,
    u'train_path': u'data/splits/ml-100k/sparse-item/0.7-train.csv',
    u'train_tes

In [7]:
# second best config
with open('../models/mfnn/2016-05-30_19.15.45_mfnn_ml-100k_e20_tt-0.7_task-6_config.json') as f:
    pp.pprint(json.loads(f.read()))

{   u'ada_eps': 1e-06,
    u'adagrad': True,
    u'binarize': True,
    u'binarize_neg': 0,
    u'binarize_pos': 1,
    u'binarize_threshold': 1,
    u'd2v_model': u'doc2vec-models/2016-04-14_17.36.08_20e_pv-dbow_size50_lr0.025_window8_neg5',
    u'eval_in_parallel': False,
    u'experiment_name': u'mfnn_ml-100k_e20_tt-0.7_task-6',
    u'hit_threshold': 4,
    u'lr': 0.03,
    u'lr_decay': 0.0005,
    u'metrics_save_dir': u'metrics/mfnn',
    u'model_save_dir': u'models/mfnn',
    u'nb_d2v_features': 50,
    u'nb_epochs': 20,
    u'nb_latent_f': 96,
    u'nb_movies': 1261,
    u'nb_users': 943,
    u'precision_recall_at_n': 20,
    u'ratings_path': u'data/splits/ml-100k/ratings.csv',
    u'reg_lambda': 0.01,
    u'run_eval': True,
    u'run_movie_metrics': False,
    u'sparse_item': True,
    u'test': True,
    u'test_path': u'data/splits/ml-100k/sparse-item/0.7-test.csv',
    u'top_n_predictions': 100,
    u'train_path': u'data/splits/ml-100k/sparse-item/0.7-train.csv',
    u'train_te

## ML-1M - 0.7 train / 0.3 test - sparse item

In [ ]:
results = [

]

In [ ]:
result_map = map(lambda p: (p, pd.read_csv(p)['auc'].mean()), results)
sorted(result_map, key=lambda t:t[1], reverse=True)

In [ ]:
# best config
with open('../models/mfnn/_config.json') as f:
    pp.pprint(json.loads(f.read()))

In [ ]:
best_metrics = pd.read_csv('../metrics/mfnn/_user-metrics.csv')
best_metrics.mean(0)

## ML-1m - 20e - 0.2 train/0.8 test

In [2]:
results = [
'../metrics/mfnn/2016-05-13_13.07.07_mfnn_ml-1m_e20_tt-0.2_task-19_user-metrics.csv',
'../metrics/mfnn/2016-05-13_13.07.24_mfnn_ml-1m_e20_tt-0.2_task-18_user-metrics.csv',
'../metrics/mfnn/2016-05-13_13.08.14_mfnn_ml-1m_e20_tt-0.2_task-3_user-metrics.csv',
'../metrics/mfnn/2016-05-13_13.08.42_mfnn_ml-1m_e20_tt-0.2_task-11_user-metrics.csv',
'../metrics/mfnn/2016-05-13_13.08.49_mfnn_ml-1m_e20_tt-0.2_task-5_user-metrics.csv',
'../metrics/mfnn/2016-05-13_13.08.59_mfnn_ml-1m_e20_tt-0.2_task-1_user-metrics.csv',
'../metrics/mfnn/2016-05-13_13.13.32_mfnn_ml-1m_e20_tt-0.2_task-0_user-metrics.csv',
'../metrics/mfnn/2016-05-13_14.02.05_mfnn_ml-1m_e20_tt-0.2_task-7_user-metrics.csv',
'../metrics/mfnn/2016-05-13_14.19.45_mfnn_ml-1m_e20_tt-0.2_task-13_user-metrics.csv',
'../metrics/mfnn/2016-05-13_14.22.53_mfnn_ml-1m_e20_tt-0.2_task-12_user-metrics.csv',
'../metrics/mfnn/2016-05-13_14.54.52_mfnn_ml-1m_e20_tt-0.2_task-17_user-metrics.csv',
'../metrics/mfnn/2016-05-13_16.17.19_mfnn_ml-1m_e20_tt-0.2_task-6_user-metrics.csv',
'../metrics/mfnn/2016-05-13_16.30.09_mfnn_ml-1m_e20_tt-0.2_task-20_user-metrics.csv',
'../metrics/mfnn/2016-05-13_16.36.33_mfnn_ml-1m_e20_tt-0.2_task-14_user-metrics.csv',
'../metrics/mfnn/2016-05-13_17.12.47_mfnn_ml-1m_e20_tt-0.2_task-21_user-metrics.csv',
'../metrics/mfnn/2016-05-13_17.53.02_mfnn_ml-1m_e20_tt-0.2_task-8_user-metrics.csv',
'../metrics/mfnn/2016-05-13_18.35.07_mfnn_ml-1m_e20_tt-0.2_task-23_user-metrics.csv',
'../metrics/mfnn/2016-05-13_19.30.42_mfnn_ml-1m_e20_tt-0.2_task-10_user-metrics.csv',
'../metrics/mfnn/2016-05-13_20.08.12_mfnn_ml-1m_e20_tt-0.2_task-22_user-metrics.csv',
'../metrics/mfnn/2016-05-13_20.10.52_mfnn_ml-1m_e20_tt-0.2_task-15_user-metrics.csv',
'../metrics/mfnn/2016-05-13_22.18.50_mfnn_ml-1m_e20_tt-0.2_task-2_user-metrics.csv',
]

In [3]:
result_map = map(lambda p: (p, pd.read_csv(p)['auc'].mean()), results)
sorted(result_map, key=lambda t:t[1], reverse=True)

[('../metrics/mfnn/2016-05-13_16.17.19_mfnn_ml-1m_e20_tt-0.2_task-6_user-metrics.csv',
  0.8978495667178319),
 ('../metrics/mfnn/2016-05-13_19.30.42_mfnn_ml-1m_e20_tt-0.2_task-10_user-metrics.csv',
  0.8911205847799619),
 ('../metrics/mfnn/2016-05-13_13.08.49_mfnn_ml-1m_e20_tt-0.2_task-5_user-metrics.csv',
  0.8889464453164606),
 ('../metrics/mfnn/2016-05-13_13.08.42_mfnn_ml-1m_e20_tt-0.2_task-11_user-metrics.csv',
  0.8830935640725851),
 ('../metrics/mfnn/2016-05-13_13.13.32_mfnn_ml-1m_e20_tt-0.2_task-0_user-metrics.csv',
  0.8804266743776397),
 ('../metrics/mfnn/2016-05-13_22.18.50_mfnn_ml-1m_e20_tt-0.2_task-2_user-metrics.csv',
  0.8781281670501629),
 ('../metrics/mfnn/2016-05-13_20.10.52_mfnn_ml-1m_e20_tt-0.2_task-15_user-metrics.csv',
  0.8779493811055779),
 ('../metrics/mfnn/2016-05-13_14.19.45_mfnn_ml-1m_e20_tt-0.2_task-13_user-metrics.csv',
  0.8776865504778436),
 ('../metrics/mfnn/2016-05-13_17.53.02_mfnn_ml-1m_e20_tt-0.2_task-8_user-metrics.csv',
  0.8774628776136015),
 ('../

In [4]:
# best config
with open('../models/mfnn/2016-05-13_16.17.19_mfnn_ml-1m_e20_tt-0.2_task-6_config.json') as f:
    pp.pprint(json.loads(f.read()))

{   u'ada_eps': 1e-06,
    u'adagrad': True,
    u'binarize': True,
    u'binarize_neg': 0,
    u'binarize_pos': 1,
    u'binarize_threshold': 1,
    u'd2v_model': u'doc2vec-models/2016-04-14_17.36.08_20e_pv-dbow_size50_lr0.025_window8_neg5',
    u'eval_in_parallel': False,
    u'experiment_name': u'mfnn_ml-1m_e20_tt-0.2_task-6',
    u'hit_threshold': 4,
    u'lr': 0.01,
    u'lr_decay': 0.02,
    u'metrics_save_dir': u'metrics/mfnn',
    u'model_save_dir': u'models/mfnn',
    u'nb_d2v_features': 50,
    u'nb_epochs': 20,
    u'nb_latent_f': 96,
    u'precision_recall_at_n': 20,
    u'ratings_path': u'data/splits/ml-1m/ratings.csv',
    u'reg_lambda': 0.01,
    u'run_eval': True,
    u'run_movie_metrics': False,
    u'sparse_item': True,
    u'test': True,
    u'test_path': u'data/splits/ml-1m/sparse-item/0.2-test.csv',
    u'top_n_predictions': 100,
    u'train_path': u'data/splits/ml-1m/sparse-item/0.2-train.csv',
    u'train_test_split': 0.2,
    u'use_avg_rating': False,
    u'user

In [5]:
with open('../models/mfnn/2016-05-13_19.30.42_mfnn_ml-1m_e20_tt-0.2_task-10_config.json') as f:
    pp.pprint(json.loads(f.read()))

{   u'ada_eps': 1e-06,
    u'adagrad': True,
    u'binarize': True,
    u'binarize_neg': 0,
    u'binarize_pos': 1,
    u'binarize_threshold': 1,
    u'd2v_model': u'doc2vec-models/2016-04-14_17.36.08_20e_pv-dbow_size50_lr0.025_window8_neg5',
    u'eval_in_parallel': False,
    u'experiment_name': u'mfnn_ml-1m_e20_tt-0.2_task-10',
    u'hit_threshold': 4,
    u'lr': 0.03,
    u'lr_decay': 0.02,
    u'metrics_save_dir': u'metrics/mfnn',
    u'model_save_dir': u'models/mfnn',
    u'nb_d2v_features': 50,
    u'nb_epochs': 20,
    u'nb_latent_f': 128,
    u'precision_recall_at_n': 20,
    u'ratings_path': u'data/splits/ml-1m/ratings.csv',
    u'reg_lambda': 0.01,
    u'run_eval': True,
    u'run_movie_metrics': False,
    u'sparse_item': True,
    u'test': True,
    u'test_path': u'data/splits/ml-1m/sparse-item/0.2-test.csv',
    u'top_n_predictions': 100,
    u'train_path': u'data/splits/ml-1m/sparse-item/0.2-train.csv',
    u'train_test_split': 0.2,
    u'use_avg_rating': True,
    u'use

In [6]:
with open('../models/mfnn/2016-05-13_13.08.49_mfnn_ml-1m_e20_tt-0.2_task-5_config.json') as f:
    pp.pprint(json.loads(f.read()))

{   u'ada_eps': 1e-06,
    u'adagrad': True,
    u'binarize': True,
    u'binarize_neg': 0,
    u'binarize_pos': 1,
    u'binarize_threshold': 1,
    u'd2v_model': u'doc2vec-models/2016-04-14_17.36.08_20e_pv-dbow_size50_lr0.025_window8_neg5',
    u'eval_in_parallel': False,
    u'experiment_name': u'mfnn_ml-1m_e20_tt-0.2_task-5',
    u'hit_threshold': 4,
    u'lr': 0.01,
    u'lr_decay': 0.02,
    u'metrics_save_dir': u'metrics/mfnn',
    u'model_save_dir': u'models/mfnn',
    u'nb_d2v_features': 50,
    u'nb_epochs': 20,
    u'nb_latent_f': 64,
    u'precision_recall_at_n': 20,
    u'ratings_path': u'data/splits/ml-1m/ratings.csv',
    u'reg_lambda': 0.003,
    u'run_eval': True,
    u'run_movie_metrics': False,
    u'sparse_item': True,
    u'test': True,
    u'test_path': u'data/splits/ml-1m/sparse-item/0.2-test.csv',
    u'top_n_predictions': 100,
    u'train_path': u'data/splits/ml-1m/sparse-item/0.2-train.csv',
    u'train_test_split': 0.2,
    u'use_avg_rating': False,
    u'use